In [1]:
import json
import pandas as pd
import numpy as np
from config import username
from config import passwd
from sqlalchemy import create_engine, inspect

In [2]:
# File path
business_json = 'data/yelp_academic_dataset_business.json'

In [3]:
# Read JSON file and extract raw data
business_df = pd.read_json(business_json, lines=True)
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
# Make a copy and drop unneeded columns
clean_df = business_df[['name', 'address', 'city', 'state', 'postal_code', 'stars', 'review_count', 'categories']].copy().dropna()
clean_df.head()

,name,address,city,state,postal_code,stars,review_count,categories
0,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B..."
1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
2,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,4.5,13,"Antiques, Fashion, Used, Vintage & Consignment..."
3,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,3.0,8,"Beauty & Spas, Hair Salons"
4,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,4.0,14,"Gyms, Active Life, Interval Training Gyms, Fit..."


In [5]:
# Filter for 'restaurants' or 'coffee & tea'
keywords = ["Restaurants", "Coffee & Tea"]
clean_df = clean_df[clean_df['categories'].str.contains('|'.join(keywords))]
clean_df.head()

,name,address,city,state,postal_code,stars,review_count,categories
0,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B..."
1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
5,Bob Likes Thai Food,3755 Main St,Vancouver,BC,V5V,3.5,169,"Restaurants, Thai"
7,Boxwood Biscuit,740 S High St,Columbus,OH,43206,4.5,11,"Breakfast & Brunch, Restaurants"
12,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,4.0,39,"Food, Pizza, Restaurants"


In [6]:
# Reset the index
clean_yelp_df = clean_df.reset_index(drop=True)
clean_yelp_df

,name,address,city,state,postal_code,stars,review_count,categories
0,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B..."
1,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C..."
2,Bob Likes Thai Food,3755 Main St,Vancouver,BC,V5V,3.5,169,"Restaurants, Thai"
3,Boxwood Biscuit,740 S High St,Columbus,OH,43206,4.5,11,"Breakfast & Brunch, Restaurants"
4,Mr G's Pizza & Subs,474 Lowell St,Peabody,MA,01960,4.0,39,"Food, Pizza, Restaurants"
...,...,...,...,...,...,...,...,...
54094,Split Dine and Drink,7335 SW Bridgeport Rd,Tigard,OR,97224,2.0,48,"American (Traditional), Desserts, American (Ne..."
54095,Maudie’s Hill Country,12506 Shops Pkwy,Bee Cave,TX,78738,3.0,310,"Mexican, Bars, Nightlife, Tex-Mex, Restaurants"
54096,Mama's Cocina Latina,1958 Piedmont Rd NE,Atlanta,GA,30324,3.0,185,"Restaurants, Mexican, Latin American"
54097,Havana Cafe,910 NW 14th St,Portland,OR,97209,4.0,10,"Cuban, Sandwiches, Restaurants, Cafes"
